In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

'''
模型
初始有100W$
         总池：98W $  初始汇率：7，汇率的涨降幅度一定是0.001
       美元池：1W  $  池子中每一份钱都有对应买入时的成本（汇率）
     人民币池：7W ￥  池子中每一份钱都有对应买入时的成本（汇率）
操作的每一份都是固定额度的钱（这里定为1W$）
操作1：将美元池的钱卖到人民币池中
          当汇率上涨时，计算当前汇率与美元池中的每一份钱的成本（汇率）的比，将>=1.005的“份”卖到人民币池中
操作2：将人民币中的钱卖到美元池中
          当汇率下降时，计算当前汇率与人民币池中的每一份钱的成本（汇率）的比，将<=0.995的“份”卖到美元池中
操作3：从总池中买一份到美元池中
          当汇率下降时

TODO
1. 汇率涨浮是百分比，而不是绝对值
2. 操作3
3. 模拟更真实的汇率变化，例如40%是上涨，60%是下跌，整体成下跌趋势
'''

'\n模型\n初始有100W$\n         总池：98W $  初始汇率：7，汇率的涨降幅度一定是0.001\n       美元池：1W  $  池子中每一份钱都有对应买入时的成本（汇率）\n     人民币池：7W ￥  池子中每一份钱都有对应买入时的成本（汇率）\n操作的每一份都是固定额度的钱（这里定为1W$）\n操作1：将美元池的钱卖到人民币池中\n          当汇率上涨时，计算当前汇率与美元池中的每一份钱的成本（汇率）的比，将>=1.005的“份”卖到人民币池中\n操作2：将人民币中的钱卖到美元池中\n          当汇率下降时，计算当前汇率与人民币池中的每一份钱的成本（汇率）的比，将<=0.995的“份”卖到美元池中\n操作3：从总池中买一份到美元池中\n          当汇率下降时\n\nTODO\n1. 汇率涨浮是百分比，而不是绝对值\n2. 操作3\n3. 模拟更真实的汇率变化，例如40%是上涨，60%是下跌，整体成下跌趋势\n'

In [2]:
def simulation(n):
    rate_init = 7.0
    step_len = 1000

    # ================ 计算使用的汇率 ================
    draws = np.random.randint(0, 10, size=step_len-1)
    step_value = np.where(draws>5, 1.001, 0.999); step_value = np.insert(step_value, 0, [1, 1])  # 涨：40%，降60%
    step_bool = np.where(draws>5, True, False); step_bool = np.insert(step_bool, 0, [np.NaN, np.NAN])  # True: 涨, False: 跌
    step_symbo = np.where(step_bool, '↑', '↓')
    step_rate = step_value.cumprod() * rate_init
    # ================ 计算使用的汇率 ================

    # ================ 计算是否从总池子中交易美元至美元池中 ================
    rate_pool = rate_init
    step_rate_pool = np.array(rate_init)
    step_pool_ex = [np.NaN, np.NaN]
    for i in range(2, step_len+1):
        if not step_bool[i]:  # 降
            if (step_rate[i]/rate_pool) <= 0.995:
                rate_pool = step_rate[i]
                step_rate_pool = np.append(step_rate_pool, step_rate[i])
                step_pool_ex.append(True)
            else:
                step_pool_ex.append(False)
        else:                 # 涨
            if step_rate[i] > rate_pool:
                rate_pool = step_rate_pool[step_rate_pool<step_rate[i]].max()
            step_pool_ex.append(False)
    # ================ 计算是否从总池子中交易美元至美元池中 ================

    unit = 10000.
    pool = [unit*100, unit*98]
    usd = [0, unit]
    rmb = [7, unit*rate_init]

    usd_cur = [(unit, 7)]
    rmb_cur = [(unit*rate_init, 7)]
    step_usd = [[], [(unit, 7)]]
    step_rmb = [[], [(unit*rate_init, 7)]]

    for i in range(2, step_len+1):  # 第零次初始值，第一次相当于从总池子中分别买入一份美元与一份人民币，所以从第二次开始
        # 美元池与人民币池的交易
        flag_ex = False
        usd_ex = 0.
        rmb_ex = 0.
        if step_bool[i]:  # 涨
            for item in usd_cur[::-1]:
                if (step_rate[i]/item[1]) >= 1.005:
                    flag_ex = True
                    rmb_ex += item[0]*step_rate[i]
                    rmb_cur.append((item[0]*step_rate[i], step_rate[i]))
                    usd_ex += item[0]
                    usd_cur.remove(item)
            if flag_ex:
                step_rmb.append(rmb_cur.copy())
                rmb.append(rmb[-1]+rmb_ex)
                step_usd.append(usd_cur.copy())
                usd.append(usd[-1]-usd_ex)
            else:
                step_rmb.append(step_rmb[-1].copy())
                rmb.append(rmb[-1])
                step_usd.append(step_usd[-1].copy())
                usd.append(usd[-1])
        else:            # 降
            for item in rmb_cur[::-1]:
                if (step_rate[i]/item[1]) <= 0.995:
                    flag_ex = True
                    usd_ex += item[0]/step_rate[i]
                    usd_cur.append((item[0]/step_rate[i], step_rate[i]))  
                    rmb_ex += item[0]
                    rmb_cur.remove(item)
            if flag_ex:
                step_usd.append(usd_cur.copy())
                usd.append(usd[-1]+usd_ex)
                step_rmb.append(rmb_cur.copy())
                rmb.append(rmb[-1]-rmb_ex)
            else:
                step_usd.append(step_usd[-1].copy())
                usd.append(usd[-1])
                step_rmb.append(step_rmb[-1].copy())
                rmb.append(rmb[-1])

        # 总池与美元池的交易
        if step_pool_ex[i]:
            pool.append(pool[-1]-unit)
            usd[-1] += unit
            usd_cur.append((unit, step_rate[i]))
            step_usd[-1].append((unit, step_rate[i]))
        else:
            pool.append(pool[-1])  # 总池不变

    total = list(np.array(pool)+np.array(usd)+np.array(rmb)/np.array(step_rate))  # TODO: 人民币还没有除汇率

    result = pd.DataFrame({'↑↓':step_symbo,
                           'Rate':step_rate,
                           'Pool_ex':step_pool_ex,
                           'Pool':pool,
                           'USD':usd,
                           'RMB':rmb,
                           'Total':total,
                           'USD_Detail':step_usd,
                           'RMB_Detail':step_rmb})
    result.to_excel(f'data/Model2-{n}.xlsx')
    # result

In [3]:
! mkdir -p data

for i in range(10):
    simulation(i)

In [4]:
!rm data/Model2-*.xlsx